In [34]:
#import
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import scipy.cluster.hierarchy as hr
from geojson import LineString
from geojson import MultiLineString
from shapely.geometry import MultiLineString
import random
import math
import os
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import pairwise_distances
import pandas as pd
import folium
from shapely import wkt, geometry
import json
from pprint import pprint
from openrouteservice import client, places

api_key = '5b3ce3597851110001cf62487a4f3ad33ea14791b72150c9e6446e5e'
clnt = client.Client(key=api_key)

In [35]:
#GPS data open and read > path(gps_dummy) generations
x=0
paths ='C:/Users/찬란한 돌멩이/Desktop/GPS DAT/Geolife Trajectories 1.3/Geolife Trajectories 1.3/Data/000/Trajectory'
for filename in os.listdir(paths):
    a=[]
    with open(os.path.join(paths, filename), 'r') as f:
        #print(os.path.join(paths, filename))
        lines = f.readlines()[9:]
        for i in lines:
            c= i.split(sep=',')
            if len(c)>=7:
                b= i.split(sep=',')
                b[6]=b[6].replace("\n","")
                b[6]=b[6].replace(":","")
                a.append(b)
    n_a = np.array(a)
    for i in range(4):
        n_a = np.delete(n_a,2,axis=1)
    n_a = n_a.astype(np.float64)
    globals()['GPS_dummy_'+str(x)] = n_a.tolist()
    x+=1


        

In [44]:
#DTW분석
distance_matrix_dtw=np.zeros(shape=(x,x))
path_list=[]


for i in range(x-1):

    distance, path = fastdtw(globals()['GPS_dummy_'+str(i)],globals()['GPS_dummy_'+str(i+1)] , dist=euclidean)
    distance_matrix_dtw[i][i+1] = distance
    path_list.append(path) 
distance_matrix_dtw = distance_matrix_dtw.tolist()

In [45]:
#Agglomerative clustering
n_clusters=12
agg = AgglomerativeClustering(n_clusters= n_clusters, metric='precomputed',
                              linkage='average')
u = agg.fit_predict(distance_matrix_dtw)

In [46]:
#loading Agglomerative Clustering result on differnt lists(클러스터 수만큼의 리스트들)

a=0
b=0
for i in range(n_clusters):
    globals()['Clusters_'+str(i)]=[]
    temp=[]
    
    for j in u:
        if j == a:
            temp.append(b)
        b+=1
    b=0
    globals()['Clusters_'+str(i)] = temp
    a+=1

In [47]:
#initializing_1
for i in range(x):
    globals()['temp_'+str(i)]=globals()['GPS_dummy_'+str(i)]

In [48]:
#initializing_2
for i in range(x):
    globals()['GPS_dummy_'+str(i)]=globals()['temp_'+str(i)]

#based on Clustering results, Making Clustered GPS path lists.
for i in range(n_clusters):
    globals()['Clusters_path_'+str(i)]=[]
    for j in globals()['Clusters_'+str(i)]:
        globals()['GPS_dummy_'+str(j)] = np.array(globals()['GPS_dummy_'+str(j)])
        globals()['GPS_dummy_'+str(j)] = np.delete(globals()['GPS_dummy_'+str(j)],2,1)
        globals()['GPS_dummy_'+str(j)][:,[0,1]] = globals()['GPS_dummy_'+str(j)][:,[1,0]] 
        globals()['Clusters_path_'+str(i)].append(globals()['GPS_dummy_'+str(j)])


In [49]:
#making path polygons[ith_cluster][jth_path]
for i in range(n_clusters):
    for j in range(len(globals()['Clusters_path_'+str(i)])):
        globals()['path_polygon_'+str(i)+str(j)]=[]
        for k in globals()['Clusters_path_'+str(i)][j]:
            globals()['path_polygon_'+str(i)+str(j)].append(str(k[0])+" "+str(k[1])+" ")
        globals()['path_polygon_'+str(i)+str(j)] = ','.join(globals()['path_polygon_'+str(i)+str(j)])
        globals()['path_polygon_'+str(i)+str(j)]= 'LineString'+" "+"("+ globals()['path_polygon_'+str(i)+str(j)] +")"

In [92]:
#mapping paths of a cluster randomly chosen
what_cluster=random.randint(0,n_clusters-1)

wkt_str = globals()['path_polygon_'+str(what_cluster)+str(len(globals()['Clusters_path_'+str(what_cluster)])-1)]
aoi_geom = wkt.loads(wkt_str)

aoi_coords = list(aoi_geom.coords)
aoi_coords = [(y,x) for x,y in aoi_coords] 
aoi_centroid = aoi_geom.centroid 


m = folium.Map(location=(aoi_centroid.y, aoi_centroid.x), zoom_start=16)

for i in range(0,len(globals()['Clusters_path_'+str(what_cluster)])):
    globals()['wkt_str_'+str(i)] = globals()['path_polygon_'+str(what_cluster)+str(i)]
    globals()['aoi_geom_'+str(i)] = wkt.loads(globals()['wkt_str_'+str(i)])
    globals()['aoi_coords_'+str(i)] = list(globals()['aoi_geom_'+str(i)].coords)
    globals()['aoi_coords_'+str(i)] = [(y,x) for x,y in globals()['aoi_coords_'+str(i)]] 
    globals()['aoi_centroid'+str(i)]= globals()['aoi_geom_'+str(i)].centroid
    
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]

    folium.PolyLine(globals()['aoi_coords_'+str(i)],
                                     color=color[0],
                                     #fill_color=color[0],
                                     fill_opacity=0.2,
                                     weight=3).add_to(m)
   

m

In [53]:
#extracting all the index numbers of summarized paths of each clusters.
index_for_summarized_path_l=[]
for i in range(n_clusters):
     if len(globals()['Clusters_path_'+str(i)])>2:
          globals()['path_score_count_'+str(i)]=[0 for j in range(len(globals()['Clusters_path_'+str(i)]))]
          for a in range(len(globals()['Clusters_path_'+str(i)])-1):
               for b in range(a+1, len(globals()['Clusters_path_'+str(i)])):
                    distance, path = fastdtw(globals()['Clusters_path_'+str(i)][a],globals()['Clusters_path_'+str(i)][b] , dist=euclidean)
                    globals()['path_score_count_'+str(i)][a]+=distance/2
                    globals()['path_score_count_'+str(i)][b]+=distance/2
                    
          #index_for_summarized_path[i] = globals()['path_score_count_'+str(i)].index(min(globals()['path_score_count_'+str(i)]))
          index_for_summarized_path_l.append(globals()['path_score_count_'+str(i)].index(min(globals()['path_score_count_'+str(i)])))
               
     if len(globals()['Clusters_path_'+str(i)])==2:
          if len(globals()['Clusters_path_'+str(i)][0]) >= len(globals()['Clusters_path_'+str(i)][1]):
               index_for_summarized_path_l.append(1)
                    
          if len(globals()['Clusters_path_'+str(i)][0]) < len(globals()['Clusters_path_'+str(i)][1]):
               index_for_summarized_path_l.append(0)
                    
                         
     if len(globals()['Clusters_path_'+str(i)])==1:
          index_for_summarized_path_l.append(0)


In [54]:
##making summarized paths for each clusters
for x in range(0,n_clusters):
    globals()['Summarized_path_of_Clusters_path_'+str(x)]= globals()['Clusters_path_'+str(x)][index_for_summarized_path_l[x]]

In [55]:
#converting a summarized path of clusters into shapely polygon format.
for j in range(n_clusters):    
    globals()['path_polygon_summarized_'+str(j)]=[]
    for i in globals()['Summarized_path_of_Clusters_path_'+str(j)]:
        globals()['path_polygon_summarized_'+str(j)].append(str(i[0])+" "+str(i[1])+" ")
    globals()['path_polygon_summarized_'+str(j)]= ','.join(globals()['path_polygon_summarized_'+str(j)])
    globals()['path_polygon_summarized_'+str(j)]= 'LineString'+" "+"("+globals()['path_polygon_summarized_'+str(j)]+")"

In [57]:
#mapping all summarized paths on openroutemap
what_cluster=random.randint(0,n_clusters-1)

wkt_str = globals()['path_polygon_summarized_'+str(what_cluster)]
aoi_geom = wkt.loads(wkt_str)
aoi_centroid = aoi_geom.centroid 


m = folium.Map(location=(aoi_centroid.y, aoi_centroid.x), zoom_start=16)

for i in range(n_clusters):
    if len(globals()['path_polygon_summarized_'+str(i)])>=20:
        globals()['wkt_str_'+str(i)] = globals()['path_polygon_summarized_'+str(i)]
        globals()['aoi_geom_'+str(i)] = wkt.loads(globals()['wkt_str_'+str(i)])
        globals()['aoi_coords_'+str(i)] = list(globals()['aoi_geom_'+str(i)].coords)
        globals()['aoi_coords_'+str(i)] = [(y,x) for x,y in globals()['aoi_coords_'+str(i)]] 
        globals()['aoi_centroid_'+str(i)]= globals()['aoi_geom_'+str(i)].centroid
        
        color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]

        folium.PolyLine(globals()['aoi_coords_'+str(i)],
                                        color=color[0],
                                        #fill_color=color[0],
                                        fill_opacity=0.2,
                                        weight=3).add_to(m)
m
   